<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

# 실습 내용

- Grid Search로 KNN 알고리즘 모델을 튜닝합니다.


# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [ ]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%config InlineBackend.figure_format='retina'

In [ ]:
# 데이터 읽어오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/admission_simple.csv'
data = pd.read_csv(path)

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.

In [ ]:
# 상위 몇 개 행 확인
data.head()

In [ ]:
# 변수 확인
data.info()

In [ ]:
# 기초통계량 확인
data.describe()

# 3.데이터 준비

- 전처리 과정을 통해 머신러닝 알고리즘에 사용할 수 있는 형태의 데이터를 준비합니다.

**1) 가변수화**

In [ ]:
# 가변수화
cols = ['RANK']
data = pd.get_dummies(data, columns=cols, drop_first=True)

**2) x, y 분리**

In [ ]:
# target 확인
target = 'ADMIT'

# 데이터 분리
x = data.drop(target, axis = 1)
y = data[target] 

**3) 정규화**

In [ ]:
# 정규화
cols = ['GRE', 'TOEFL', 'SOP', 'LOR', 'GPA']
x[cols] = (x[cols] - x[cols].min()) / (x[cols].max() - x[cols].min())

# 확인
x.head()

**4) 학습용, 평가용 데이터 분리**

In [ ]:
# 모듈 불러오기
from sklearn.model_selection import train_test_split

# 7:3으로 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2022)

# 4.모델링

- 본격적으로 모델을 선언하고 학습하고 평가하는 과정을 진행합니다.

In [ ]:
# 1단계: 불러오기
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# 2단계: 선언하기
model = KNeighborsClassifier(n_neighbors=5)

In [ ]:
# 3단계: 학습하기
model.fit(x_train, y_train)

In [ ]:
# 4단계: 예측하기
y_pred = model.predict(x_test)

In [ ]:
# 5단계: 평가하기
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

**성능에 대한 고민**

- 위 모델링 과정에서 n_neighbors=5를 지정했습니다.
- n_neighbors를 다른 값으로 지정하면 성능이 더 좋을 수도 있습니다.
- 경험이 없는 상태에서 최적의 파라미터를 지정하기는 쉽지 않습니다.

# 5.성능 튜닝

- GridSearchCV를 사용해 최선의 하이퍼파라미터를 갖는 모델을 만듭니다.

**1) 모델 만들기**

In [ ]:
# 1단계: 불러오기
f



- 다음 두 가지 모델을 선언합니다.
    - 기본 모델: 기본 알고리즘을 사용하는 튜닝 대상 모델
    - 최종 모델: GridSearchCV 알고리즘을 사용하는 모델
- 다음 정보를 최종 모델에 파라미터로 전달합니다.
    - 기본 모델 이름
    - 파라미터 변수
    - cv: K-Fold 분할 대수(기본값=5)
    - scoring: 평가 방법

In [ ]:
# 1단계: 선언하기


# 파라미터 선언
  # n_neighbors: range(5, 51)

    
# Grid Search 모델 선언하기
  # cv=10
  # scoring='accuracy'





In [ ]:
# 2단계: 학습하기


**2) 결과 확인**

- model.cv_results_ 속성에 성능 테스트와 관련된 많은 정보가 포함되어 있습니다.
- 데이터프레임으로 만들어 확인하면 좋습니다.

In [ ]:
# 데이터프레임 만들기



In [ ]:
# 중요 정보 확인



- model.best_params_, model.best_score_ 값이 최적의 파라미터, 최고의 학습성능입니다.

In [ ]:
# 선택된 파라미터와 성능 확인



- 성능 정보를 시각화해 파라미터에 따른 성능 변화를 확인합니다.

In [ ]:
# 최적파라미터 값
print('최적파라미터:', model.best_params_)

# 시각화
plt.figure(figsize=(10, 5))
plt.plot(result['param_n_neighbors'], result['mean_test_score'])
plt.axvline(model.best_params_['n_neighbors'], color='r')
plt.xlabel('N_Neighbors')
plt.ylabel('Score')
plt.show()

**3) 예측과 평가**

- 학습을 통해 예상한 성능과 실제 평가에 따른 성능은 차이가 있을 수 있습니다.
- 예선전에서 성적이 좋았다고 본선에서도 성적이 좋다고 보장할 수 없는 없겠지요?

In [ ]:
# 4단계: 예측하기


In [ ]:
# 5단계: 평가하기

